<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#IEEE-CIS-Fraud-Detection" data-toc-modified-id="IEEE-CIS-Fraud-Detection-1">IEEE-CIS Fraud Detection</a></span><ul class="toc-item"><li><span><a href="#Imports" data-toc-modified-id="Imports-1.1">Imports</a></span></li><li><span><a href="#Reading-in-and-merging-dataframes" data-toc-modified-id="Reading-in-and-merging-dataframes-1.2">Reading in and merging dataframes</a></span></li><li><span><a href="#Data-exploration-and-preprocessing" data-toc-modified-id="Data-exploration-and-preprocessing-1.3">Data exploration and preprocessing</a></span></li><li><span><a href="#Creating-simplified-data-sets-for--experimentation" data-toc-modified-id="Creating-simplified-data-sets-for--experimentation-1.4">Creating simplified data sets for  experimentation</a></span><ul class="toc-item"><li><span><a href="#Random-sampling-of-the-test-set" data-toc-modified-id="Random-sampling-of-the-test-set-1.4.1">Random sampling of the test set</a></span></li><li><span><a href="#Simplfication-of-data-set" data-toc-modified-id="Simplfication-of-data-set-1.4.2">Simplfication of data set</a></span></li></ul></li></ul></li></ul></div>

# IEEE-CIS Fraud Detection

## Imports

In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
import sys
from zipfile import ZipFile
from pathlib import Path
import pandas as pd
import multiprocessing as mp
import torch
from functools import partial
pd.options.display.max_columns = None

In [3]:
path = Path('/Users/baranserajelahi/Codes/fraud-detection-pytorch-scikit-fastai/data')

In [4]:
Path.BASE_PATH = path

In [5]:
# with ZipFile('ieee-fraud-detection.zip', 'r') as zip_ref:
#     zip_ref.extractall(path/'Data')

## Reading in and merging dataframes

In [6]:
files = [path/'test_identity.csv', 
         path/'test_transaction.csv',
         path/'train_identity.csv',
         path/'train_transaction.csv']

In [ ]:
%%time
def read_data(file):
    return pd.read_csv(file, low_memory=False)

with mp.Pool() as pool:
    test_id, test_tr, train_id, train_tr = pool.map(read_data, files)   

In [ ]:
train = pd.merge(train_tr, train_id, on='TransactionID', how='left')
test = pd.merge(test_tr, test_id, on='TransactionID', how='left')

In [ ]:
train.to_csv(path/'train.csv')
test.to_csv(path/'test.csv')

In [5]:
train = pd.read_csv(path/'train.csv', index_col=[0], low_memory=False)
test = pd.read_csv(path/'test.csv', index_col=[0], low_memory=False)

## Data exploration and preprocessing

In [6]:
print(f'Train dataset has {train.shape[0]} rows and {train.shape[1]} columns.')
print(f'Test dataset has {test.shape[0]} rows and {test.shape[1]} columns.')

Train dataset has 590540 rows and 434 columns.
Test dataset has 506691 rows and 433 columns.


In [7]:
def nans_by_col(df):
    p_nan = {}
    print('--Lengths--' + '--NaN Counts--' + '--Percent NaN--')
    for col in df.columns:
        nan_count = len(df[col]) - df[col].count()
        length = len(df[col])
        percent_nan = nan_count/length
        p_nan[col] = percent_nan
        print(f'{col}: {length},     {nan_count},     {percent_nan}')
    return p_nan

In [8]:
p_nan_train = nans_by_col(train)

--Lengths----NaN Counts----Percent NaN--
TransactionID: 590540,     0,     0.0
isFraud: 590540,     0,     0.0
TransactionDT: 590540,     0,     0.0
TransactionAmt: 590540,     0,     0.0
ProductCD: 590540,     0,     0.0
card1: 590540,     0,     0.0
card2: 590540,     8933,     0.015126833068039422
card3: 590540,     1565,     0.0026501168422122124
card4: 590540,     1577,     0.00267043722694483
card5: 590540,     4259,     0.007212043214684865
card6: 590540,     1571,     0.0026602770345785214
addr1: 590540,     65706,     0.1112642666034477
addr2: 590540,     65706,     0.1112642666034477
dist1: 590540,     352271,     0.596523520845328
dist2: 590540,     552913,     0.9362837403054831
P_emaildomain: 590540,     94456,     0.1599485216920107
R_emaildomain: 590540,     453249,     0.7675161716395164
C1: 590540,     0,     0.0
C2: 590540,     0,     0.0
C3: 590540,     0,     0.0
C4: 590540,     0,     0.0
C5: 590540,     0,     0.0
C6: 590540,     0,     0.0
C7: 590540,     0,     

V140: 590540,     508595,     0.8612371727571375
V141: 590540,     508595,     0.8612371727571375
V142: 590540,     508595,     0.8612371727571375
V143: 590540,     508589,     0.8612270125647712
V144: 590540,     508589,     0.8612270125647712
V145: 590540,     508589,     0.8612270125647712
V146: 590540,     508595,     0.8612371727571375
V147: 590540,     508595,     0.8612371727571375
V148: 590540,     508595,     0.8612371727571375
V149: 590540,     508595,     0.8612371727571375
V150: 590540,     508589,     0.8612270125647712
V151: 590540,     508589,     0.8612270125647712
V152: 590540,     508589,     0.8612270125647712
V153: 590540,     508595,     0.8612371727571375
V154: 590540,     508595,     0.8612371727571375
V155: 590540,     508595,     0.8612371727571375
V156: 590540,     508595,     0.8612371727571375
V157: 590540,     508595,     0.8612371727571375
V158: 590540,     508595,     0.8612371727571375
V159: 590540,     508589,     0.8612270125647712
V160: 590540,     50

V323: 590540,     508189,     0.8605496664070174
V324: 590540,     508189,     0.8605496664070174
V325: 590540,     508189,     0.8605496664070174
V326: 590540,     508189,     0.8605496664070174
V327: 590540,     508189,     0.8605496664070174
V328: 590540,     508189,     0.8605496664070174
V329: 590540,     508189,     0.8605496664070174
V330: 590540,     508189,     0.8605496664070174
V331: 590540,     508189,     0.8605496664070174
V332: 590540,     508189,     0.8605496664070174
V333: 590540,     508189,     0.8605496664070174
V334: 590540,     508189,     0.8605496664070174
V335: 590540,     508189,     0.8605496664070174
V336: 590540,     508189,     0.8605496664070174
V337: 590540,     508189,     0.8605496664070174
V338: 590540,     508189,     0.8605496664070174
V339: 590540,     508189,     0.8605496664070174
id_01: 590540,     446307,     0.7557608290716971
id_02: 590540,     449668,     0.7614522301622244
id_03: 590540,     524216,     0.8876892335828225
id_04: 590540,   

In [9]:
p_nan_test = nans_by_col(test)

--Lengths----NaN Counts----Percent NaN--
TransactionID: 506691,     0,     0.0
TransactionDT: 506691,     0,     0.0
TransactionAmt: 506691,     0,     0.0
ProductCD: 506691,     0,     0.0
card1: 506691,     0,     0.0
card2: 506691,     8654,     0.017079442895176745
card3: 506691,     3002,     0.0059247154577444635
card4: 506691,     3086,     0.006090496969553436
card5: 506691,     4547,     0.008973911121373777
card6: 506691,     3007,     0.00593458340487595
addr1: 506691,     65609,     0.1294852286699389
addr2: 506691,     65609,     0.1294852286699389
dist1: 506691,     291217,     0.5747427919580178
dist2: 506691,     470255,     0.9280902956634319
P_emaildomain: 506691,     69192,     0.13655659958436206
R_emaildomain: 506691,     370821,     0.7318484046489873
C1: 506691,     3,     5.920768278891869e-06
C2: 506691,     3,     5.920768278891869e-06
C3: 506691,     3,     5.920768278891869e-06
C4: 506691,     3,     5.920768278891869e-06
C5: 506691,     3,     5.92076827889

V201: 506691,     370316,     0.7308517419887072
V202: 506691,     369957,     0.7301432233846664
V203: 506691,     369957,     0.7301432233846664
V204: 506691,     369957,     0.7301432233846664
V205: 506691,     369957,     0.7301432233846664
V206: 506691,     369957,     0.7301432233846664
V207: 506691,     369957,     0.7301432233846664
V208: 506691,     370316,     0.7308517419887072
V209: 506691,     370316,     0.7308517419887072
V210: 506691,     370316,     0.7308517419887072
V211: 506691,     369957,     0.7301432233846664
V212: 506691,     369957,     0.7301432233846664
V213: 506691,     369957,     0.7301432233846664
V214: 506691,     369957,     0.7301432233846664
V215: 506691,     369957,     0.7301432233846664
V216: 506691,     369957,     0.7301432233846664
V217: 506691,     379963,     0.7498909591841971
V218: 506691,     379963,     0.7498909591841971
V219: 506691,     379963,     0.7498909591841971
V220: 506691,     369375,     0.7289945943385614
V221: 506691,     36

id-31: 506691,     370066,     0.7303583446321328
id-32: 506691,     436020,     0.8605244616541442
id-33: 506691,     436020,     0.8605244616541442
id-34: 506691,     434516,     0.8575561831569931
id-35: 506691,     369714,     0.7296636411540761
id-36: 506691,     369714,     0.7296636411540761
id-37: 506691,     369714,     0.7296636411540761
id-38: 506691,     369714,     0.7296636411540761
DeviceType: 506691,     369760,     0.7297544262676858
DeviceInfo: 506691,     391634,     0.7729247213785128


In [12]:
train["isFraud"].mean()

0.03499000914417313

This indicates that there is class imbalance of about 1 to 30. 

## Creating simplified data sets for  experimentation

### Random sampling of the test set

Now I will create take a sample of this dataset to work with during experimentation.

In [18]:
# randomly sample the date without replacement
train_s = train.sample(frac=0.1, axis=0)   

In [19]:
assert train_s.columns.shape[0]==train.columns.shape[0] 

In [20]:
train_s.to_csv(path/'train_s.csv')

### Simplfication of data set

In [21]:
# train_s = pd.read_csv(path/'train_s.csv', index_col=[0], low_memory=False)
# test = pd.read_csv(path/'test.csv', index_col=[0], low_memory=False)

In [23]:
train_s.shape, test.shape

((59054, 434), (506691, 433))

In [15]:
train_s.columns

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5',
       ...
       'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36', 'id_37', 'id_38',
       'DeviceType', 'DeviceInfo'],
      dtype='object', length=434)

To simplify the dataset I will drop every collumn that records a Vxxx feature. There are over 339 such features many of which are highkly correllated.

In [16]:
train_ss = train_s.drop(list(train_s.filter(regex = 'V')), axis = 1)

In [17]:
assert len(train_ss.columns)==len(train_s.columns) - 339

In [18]:
train_ss.columns

Index(['TransactionID', 'isFraud', 'TransactionDT', 'TransactionAmt',
       'ProductCD', 'card1', 'card2', 'card3', 'card4', 'card5', 'card6',
       'addr1', 'addr2', 'dist1', 'dist2', 'P_emaildomain', 'R_emaildomain',
       'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11',
       'C12', 'C13', 'C14', 'D1', 'D2', 'D3', 'D4', 'D5', 'D6', 'D7', 'D8',
       'D9', 'D10', 'D11', 'D12', 'D13', 'D14', 'D15', 'M1', 'M2', 'M3', 'M4',
       'M5', 'M6', 'M7', 'M8', 'M9', 'id_01', 'id_02', 'id_03', 'id_04',
       'id_05', 'id_06', 'id_07', 'id_08', 'id_09', 'id_10', 'id_11', 'id_12',
       'id_13', 'id_14', 'id_15', 'id_16', 'id_17', 'id_18', 'id_19', 'id_20',
       'id_21', 'id_22', 'id_23', 'id_24', 'id_25', 'id_26', 'id_27', 'id_28',
       'id_29', 'id_30', 'id_31', 'id_32', 'id_33', 'id_34', 'id_35', 'id_36',
       'id_37', 'id_38', 'DeviceType', 'DeviceInfo'],
      dtype='object')

In [19]:
train_ss.to_csv(path/'train_ss.csv')